In [1]:
import pandas as pd
import traceback
from fuzzywuzzy import fuzz, process
from collections import defaultdict
import gensim
import datetime
from datetime import datetime as dt, timedelta as td
import re
import textstat
from textstat.textstat import textstat
import nltk
from itertools import chain
import pandasql as ps
# from BaseAD import CreateBaseAD,QuarterIndex ,get_quarteryr
# from qaextraction import get_analysts_from_preamble
import pandas as pd
import itertools
import time
from utils import get_date_from_preamble

%load_ext autoreload
%autoreload 2

/home/affine/anaconda3/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
set1 = pd.read_csv('/media/affine/New Volume/Chubb/Data/transcript_with_cals_1901_V1_1/transcript_with_cals_1901_V1_1.csv',sep='|')
set2 = pd.read_csv('/media/affine/New Volume/Chubb/Data/transcript_with_cals_1901_V1_1/transcript_with_cals_1901_V1_2.csv',sep='|')
set3 = pd.read_csv('/media/affine/New Volume/Chubb/Data/transcript_with_cals_1901_V1_1/transcript_with_cals_1901_V1_3.csv',sep='|')
set4 = pd.read_csv('/media/affine/New Volume/Chubb/Data/transcript_with_cals_1901_V1_1/transcript_with_cals_1901_V1_4.csv',sep='|')


In [3]:
data_raw = set1.append([set2,set3,set4])
#data_raw = set2.append([set3,set4])
# len(data_raw)
# data_raw = set2

In [6]:
del(set2)
del(set3)
del(set4)
del(set1)
gc.collect()

In [7]:
data_raw["date_earning_call"] = data_raw["date_earning_call"].apply(pd.to_datetime)
data_raw["date_earning_call"] = data_raw["date_earning_call"].apply(pd.Timestamp.date)

In [8]:
data_raw = data_raw[['input_company_name2', 'About',
       'Transcript_page_heading', 'Transcript_heading', 'Preamble',
       'transcript_after_preamble', 'date_earning_call', 
       'year', 'published_date_time', 'next_publish_date', 'def_date',
       'day_diff', 'cal_def_value', 'filing_date_2', 'is_cals']]


In [9]:
data_raw = data_raw.drop_duplicates()
#data = data.dropna(how='any')
len(data_raw)


48844

In [10]:
data_raw = data_raw[~data_raw['Preamble'].isnull()]
len(data_raw)


48844

In [11]:
#EDA KIND
# Create key at date and company level

#data_raw['date_earning_call_str'] = [str(i) for i in list(data_raw['date_earning_call'])]

data_raw['date_earning_call_str'] = data_raw['date_earning_call'].apply(str)
data_raw['key'] = data_raw['input_company_name2'] + data_raw['date_earning_call_str']
data_raw['Count'] = data_raw.groupby('key')['key'].transform('count')
#data_raw['max_published_date'] = data_raw.groupby('key')['published_date_time'].transform('max')


In [12]:
data_temp = data_raw[data_raw['Count']>1 ]
len(data_temp)
#data_temp.to_csv('check_raw.csv')
#check for these cases, one of the case is filling date is different

353

In [13]:
#check1
# #(data_temp.loc[291] == data_temp.loc[292])
# #data_temp.loc[1618] == data_temp.loc[1617]
#data_temp3.loc[5114] == data_temp3.loc[5115]

# data_temp.loc[291:292]
#data_temp.loc[1616:1618]

In [31]:
#Remove duplicates at earning date and compnay level

## Step 1 - Crete a subset of data whicch are having multiple counts at Key level 
data_temp = data_raw[data_raw['Count']>1 ]
data_temp['index'] = data_temp.index
data_temp['rank'] = data_temp.groupby('key')['index'].rank(ascending=1)
data_temp2 = data_temp[data_temp['rank'] == 1]
data_temp2 = data_temp2.drop(['rank','index'],axis = 1)
data_temp1 = data_raw[data_raw['Count']==1]
data = data_temp1.append(data_temp2,ignore_index =True)

/home/affine/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/affine/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [32]:
## Deleting intermediaries

del data_temp
del data_temp1
del data_temp2
del data_raw

gc.collect()

In [33]:
data["next_earning_date"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(-1)

# 1 quarter previous transcipts 
data["1Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(1)

#2 quarter previous transcipts 
data["2Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(2)

# 3 quarter previous transcipts 
data["3Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(3)

# 4 quarter previous transcipts 
data["4Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(4)

# 5 quarter previous transcipts 
data["5Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(5)

# 6 quarter previous transcipts 
data["6Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(6)

# 7 quarter previous transcipts 
data["7Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(7)


 

In [34]:
df = data
del(data)
gc.collect()

In [36]:
#df['executives_analysts'] = df.Preamble.apply(get_analysts_from_preamble)

df = df.rename(columns = {'input_company_name2':'Company'})
df = df.rename(columns = {'transcript_after_preamble':'QA'})
df['combined']=df['Preamble'].astype(str)+df['QA'].astype(str)
df['Size'] = [len(x) for x in df.loc[:,'combined']] 
df['Size_Preamble']= [len(x) for x in df.loc[:,'Preamble']]
df['Size_QA']= [len(x) for x in df.loc[:,'QA']]


In [40]:
# comp = list(df[df['is_cals']==1]['Company'])[0:500]
# len(comp)
# df1 = df[df['Company'].isin(comp)]
# len(df1)

500

In [47]:
from generatefeatures import TextUtils,GenerateFeatures
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
textutils = TextUtils(stopword_file='data/stopwords.csv', use_spacy=True)
generatefeatures = GenerateFeatures(textutils=textutils, lm_filepath='data/LoughranMcDonald_MasterDictionary_2016.csv')

/home/affine/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: DtypeWarning: Columns (63,108,109,110,176) have mixed types. Specify dtype option on import or set low_memory=False.
  from ipykernel import kernelapp as app


In [49]:
st = time.time()

df_q_features = df1['QA'].apply(generatefeatures.generate_features).apply(pd.Series)
df_q_features.columns = [c+'_QA' for c in df_q_features.columns]

df_p_features =  df1['Preamble'].apply(generatefeatures.generate_features).apply(pd.Series)
df_p_features.columns = [c+'_Preamble' for c in df_p_features.columns]


df_c_features = df1['combined'].apply(generatefeatures.generate_features).apply(pd.Series)
df_c_features.columns = [c+'_Combined' for c in df_c_features.columns]

ed = time.time()
ed-st

In [ ]:
df2 = pd.concat([df, df_q_features], axis=1)
df2 = pd.concat([df2, df_p_features], axis=1)
df2 = pd.concat([df2, df_c_features], axis=1)

In [ ]:
# df2.to_csv('df2.csv')

In [41]:
df3_features = df2.drop(['Transcript_page_heading', 'Transcript_heading','1Q_Back','2Q_Back','3Q_Back','4Q_Back','5Q_Back','6Q_Back'
       ,'7Q_Back','Preamble', 'QA', 'date_earning_call_str',
       'next_earning_date','About','QA_features','Preamble_features','Combined_features', 'def_date', 'filing_date_2',
       'is_cals', 'combined', 'Size', 'Size_Preamble',
],axis = 1)

In [42]:
only_features = df3_features.columns[2:]

In [44]:
def add_cols(row):
    for key in row['QA_features'].keys():
        row[key] = row['QA_features'][key]
    return row

In [45]:

df2 = df2.drop(list(only_features),axis = 1)

In [46]:
df2_temp = df2[['Company','1Q_Back','2Q_Back', '3Q_Back', '4Q_Back','5Q_Back','6Q_Back','7Q_Back','date_earning_call']]

In [48]:

for feature in only_features: 
    for quarter in ['1Q_Back','2Q_Back', '3Q_Back', '4Q_Back']:
        df2_temp = pd.merge(df2_temp,df3_features[['Company','date_earning_call',feature]],how = 'left',left_on = ['Company',quarter], 
                        right_on =['Company','date_earning_call'])

        df2_temp = df2_temp.drop(['date_earning_call_y'],axis = 1)
        df2_temp = df2_temp.rename(columns = {'date_earning_call_x':'date_earning_call'})
        old_name =only_features
        new_name =[quarter+'_'+i for i in old_name]
        cols = dict(zip(old_name,new_name))
        #     print(old_name,new_name)
        df2_temp = df2_temp.rename(columns=cols)



In [49]:
df2_temp = pd.merge(df2_temp,df2[['Company','date_earning_call','is_cals']],how = 'left',left_on = ['Company','date_earning_call'], 
                        right_on =['Company','date_earning_call'])

In [283]:
df2_temp.to_csv('Sample_AD2.csv')